In [13]:
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV



In [14]:
df=pd.read_csv(r"C:\Users\prabh\Downloads\data\listing_1.csv")

In [15]:
# Convert month column to the desired format (YYYY-MM-DD)
df['month'] = pd.to_datetime(df['month'], format='%Y-%m-%d')

In [16]:
df['data_year'] = df['month'].dt.year
df['data_month'] = df['month'].dt.month

# Drop the original datetime column
df = df.drop(columns=['month'])

In [17]:
# Drop rows with missing values
df = df.dropna(subset=['bedrooms','bathrooms_text', 'beds'])

In [18]:
# Create 'Price_Range' column
bins = [0, 150, 300, 500, 1000, df['price'].max()]
labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']
df['Price_Range'] = pd.cut(df['price'], bins=bins, labels=labels, include_lowest=True)

# Split the dataset into five DataFrames
df_1 = df[df['Price_Range'] == 'Very Low']
df_2 = df[df['Price_Range'] == 'Low']
df_3 = df[df['Price_Range'] == 'Medium']
df_4 = df[df['Price_Range'] == 'High']
df_5 = df[df['Price_Range'] == 'Very High']

# You can store these DataFrames in a dictionary for convenience
price_range_dfs = {
  'Very Low': df_1,
  'Low': df_2,
  'Medium': df_3,
  'High': df_4,
  'Very High': df_5
}

In [19]:
features = [
  'accommodates', 'bathrooms_text', 'bedrooms', 'beds',
  'minimum_nights', 'maximum_nights', 'minimum_nights_avg_ntm',
  'maximum_nights_avg_ntm', 'availability_30', 'number_of_reviews',
  'review_scores_rating', 'instant_bookable', 'calculated_host_listings_count',
  'property', 'new_host', 'host_response_score', 'host_score',
  'property_description_score', 'count_amenities'
  ,'data_year', 'data_month'
]

In [20]:
"""# Function to train and evaluate an SVR model
def train_svr(df):
    X = df[features]
    y = df['price']

    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Define hyperparameter tuning space
    param_grid = {
        'kernel': ['linear', 'rbf', 'poly'],
        'C': [1, 10, 100],
        'epsilon': [0.01, 0.1, 1]
    }

    # Initialize SVR model
    svr = svm.SVR()

    # Time Series Split: Preserve temporal ordering
    tscv = TimeSeriesSplit(n_splits=5)

    # Perform hyperparameter tuning with GridSearchCV using TimeSeriesSplit
    grid_svr = GridSearchCV(estimator=svr, param_grid=param_grid, cv=tscv, n_jobs=-1)
    grid_svr.fit(X_scaled, y)

    # Evaluate the best model on the last split (as a pseudo-test set)
    last_fold_idx = list(grid_svr.cv_results_['split0_test'].keys())[-1]
    X_last_fold = X_scaled[grid_svr.cv_results_['split0_test'][last_fold_idx]]
    y_last_fold = y[grid_svr.cv_results_['split0_test'][last_fold_idx]]
    y_pred_last_fold = grid_svr.predict(X_last_fold)

    mse = mean_squared_error(y_last_fold, y_pred_last_fold)
    mae = mean_absolute_error(y_last_fold, y_pred_last_fold)
    r2 = r2_score(y_last_fold, y_pred_last_fold)

    print(f"Best Parameters: {grid_svr.best_params_}")
    print(f"MSE: {mse:.2f}, MAE: {mae:.2f}, R2 Score: {r2:.2f}")
    print()

# Train and evaluate SVR models for each price range
for price_range, df in price_range_dfs.items():
    if not df.empty:  # Ensure the DataFrame is not empty
        print(f"--- {price_range} ---")
        train_svr(df)
    else:
        print(f"--- {price_range} ---")
        print("No data available in this price range.")
        print()
        """


'# Function to train and evaluate an SVR model\ndef train_svr(df):\n    X = df[features]\n    y = df[\'price\']\n\n    # Scale features\n    scaler = StandardScaler()\n    X_scaled = scaler.fit_transform(X)\n\n    # Define hyperparameter tuning space\n    param_grid = {\n        \'kernel\': [\'linear\', \'rbf\', \'poly\'],\n        \'C\': [1, 10, 100],\n        \'epsilon\': [0.01, 0.1, 1]\n    }\n\n    # Initialize SVR model\n    svr = svm.SVR()\n\n    # Time Series Split: Preserve temporal ordering\n    tscv = TimeSeriesSplit(n_splits=5)\n\n    # Perform hyperparameter tuning with GridSearchCV using TimeSeriesSplit\n    grid_svr = GridSearchCV(estimator=svr, param_grid=param_grid, cv=tscv, n_jobs=-1)\n    grid_svr.fit(X_scaled, y)\n\n    # Evaluate the best model on the last split (as a pseudo-test set)\n    last_fold_idx = list(grid_svr.cv_results_[\'split0_test\'].keys())[-1]\n    X_last_fold = X_scaled[grid_svr.cv_results_[\'split0_test\'][last_fold_idx]]\n    y_last_fold = y[grid_

In [21]:
# Function to train and evaluate an SVR model
def train_svr(df):
    X = df[features]
    y = df['price']

    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Initialize SVR model with default hyperparameters
    svr = svm.SVR()

    # Time Series Split: Preserve temporal ordering
    tscv = TimeSeriesSplit(n_splits=5)

    # Fit the SVR model using TimeSeriesSplit
    svr.fit(X_scaled, y)

    # Evaluate the model on the last split (as a pseudo-test set)
    last_fold_idx = list(tscv.split(X_scaled)).pop()
    X_last_fold = X_scaled[last_fold_idx[1]]
    y_last_fold = y[last_fold_idx[1]]
    y_pred_last_fold = svr.predict(X_last_fold)

    mse = mean_squared_error(y_last_fold, y_pred_last_fold)
    mae = mean_absolute_error(y_last_fold, y_pred_last_fold)
    r2 = r2_score(y_last_fold, y_pred_last_fold)

    print(f"MSE: {mse:.2f}, MAE: {mae:.2f}, R2 Score: {r2:.2f}")
    print()


In [ ]:
# Train and evaluate SVR models for each price range
for price_range, df in price_range_dfs.items():
    if not df.empty:  # Ensure the DataFrame is not empty
        print(f"--- {price_range} ---")
        train_svr(df)
    else:
        print(f"--- {price_range} ---")
        print("No data available in this price range.")
        print()

--- Very Low ---
